# Diffusion Limited Aggregation 

#### Author : B. Militzer, University of California, Berkeley 
#### Date   : Sept. 26, 2018

#### Read "The Science of Fractal Images", Ed. Peitgen and Saupe, p. 37 (1988)


In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#note, this function expects a matrix A[ix,iy] 
#and then displays so that A[:,0] is the lowest row of pixels
def display(A):
    maxX = A.shape[0]
    maxY = A.shape[1]
    B = np.zeros((maxY, maxX))
    for ix in range(0,maxX):
        for iy in range(0,maxY):
            B[maxY-1-iy,ix] = A[ix,iy]

    #Display the graphics outside of the notebook. 
    #On a PC, use '%matplotlib qt' instead.
    %matplotlib osx 
    
    plt.rcParams['figure.figsize'] = [6, 6/maxX*maxY]
    plt.imshow(B); 
    plt.axis('off'); 
    plt.show()
    plt.draw()
    plt.pause(0.01)

In [73]:
nParticles = 20000
maxX = 200 #200
maxY = 100 #100

In [74]:
# Initialize matrix containing all 2D grid points A(x,y)
# 0 <= x < maxX
# 0 <= y < maxY
# A(x,y)=0 ... site is empty
# A(x,y)>0 ... site is filled
A = np.zeros((maxX, maxY))

# Introduce a sticky wall at the bottom 
# by filling the lowest row of pixels with particles
A[:,0] = 1
print(A.transpose())

[[1. 1. 1. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [75]:
#test the display routine
display(A)

In [76]:
# To save computer time, we want to inject the new particle not too far
# above growing aggregate. We inject at on a line 'yStart', which
# keeps being increased so that it is always 'yBuffer' lines above the
# highest structure
yBuffer = 5
yStart  = 1 + yBuffer

In [ ]:
for i in range(0,nParticles):
    # Compute new starting point on the line y=yStart
    x  = np.random.randint(0,maxX)
    y  = yStart; #always start at upper limit

    while True:
        xOrg = x
        yOrg = y

        r = np.random.random(); # Random float:  0.0 <= r < 1.0
        #based on the value of 'r', move the particle
        #left, right, up, or down and change x and y accordingly
        if r <= 0.25:
            x += 1
        elif 0.25 < r <= 0.5:
            x -= 1
        elif 0.5 < r <= 0.75:
            y += 1
        else:
            y -= 1
        
        #now apply periodic boundary conditions to 'x'
        x = x % maxX
        
        if (A[x,y] == 1 or y>yStart): 
            x = xOrg
            y = yOrg
            continue; # if this site has been taken try moving in a different direction
        
        #determine the x coordionates of the left and right neighbors
        #store them in 'xm' and 'xp' and apply periodic boundary conditions again
    
        xp = (x + 1) % maxX
        xm = (x - 1) % maxX
        yp = y + 1
        ym = y - 1
        
        # To make the neighboring particle disappear, set it equal to 0
        # To make the diffusing particular disappear, do nothing ? and introduce a new particle
        
        # Intoduce a sticking probability p 
        p = 0.001
        rp = np.random.random()
        
        # Count the number of neighbors
        n = 0
        # top 
        if A[x, yp] == 1:
            n += 1
        # bot
        if A[x, ym] == 1:
            n += 1
        # right
        if A[xp, y] == 1:
            n += 1
        # left
        if A[xm, y] == 1:
            n += 1
        
        # Modify the sticking probability
        if n == 1:
            p = p
        elif n == 2:
            p *= 10
        elif n == 3:
            p *= 20
        elif n == 4:
            p *= 30
        
        # Determine if any neighboring site is occupied
        # if that is the case, enter the following 'if' clause
        #if rp <= p and (A[xp, yp] == 1 or A[xm, ym] == 1 or A[xp, ym] == 1 or A[xm, yp] == 1): 
        if n != 0 and rp <= p:
            A[x,y] = 1
            if (y+yBuffer>yStart and y+yBuffer<maxY): 
                yStart = y+yBuffer

            if (i%1000==0): 
                print(f'i= {i} \tx={x} \ty={y} \tyStart={yStart}')

            nNewParticlesPerFrame = 1000 
            if (i%nNewParticlesPerFrame==0): 
                display(A)
                
            break # particle was attached, break out of current loop and insert next one
            
    if (yStart+1==maxY): 
        print(f'Structures reached Y limit after only {i} particles')
        break

display(A)
        

i= 0 	x=66 	y=1 	yStart=6
i= 1000 	x=199 	y=8 	yStart=14
i= 2000 	x=106 	y=11 	yStart=19
i= 3000 	x=57 	y=18 	yStart=24
i= 4000 	x=158 	y=19 	yStart=30
i= 5000 	x=142 	y=26 	yStart=35
i= 6000 	x=101 	y=30 	yStart=40
i= 7000 	x=148 	y=38 	yStart=46
i= 8000 	x=145 	y=46 	yStart=52
i= 9000 	x=68 	y=49 	yStart=56
i= 10000 	x=43 	y=53 	yStart=62
